# De imagenes a vectores

Como vimos en el [Curso de ciencia de datos](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html) tenemos cajas negras que podemos entrenar mediante una tabla de entrenamiento que contiene filas con ejemplos y su respectivo resultado esperado. Estas filas son numeros que llamamos vectores. 

Que pasa si queremos entrenar una caja negra para que reconozca imagenes, por ejemplo, queremos entrenar una caja negra para que reconozca si una imagen tiene un gato o un perro. En ese caso nuestra tabla de entrenamiento estara compuesta por filas que tienen una imagen en vez de numeros como vemos a continuacion:

<div style="text-align:center">
<img src="img/tabla_perros_gatos.png" width="700"/>
</div>

Sabiendo que nuestras cajas negras funcionan muy bien con numeros en cada fila (vectores), la pregunta es como podemos hacer para convertir estas imagenes en vectores que las representen bien.

Como ya la mayoria de la gente sabe las imagenes estan compuestas por pequeños cuadraditos llamados pixeles y cada pixel tiene un numero que representa un color. Veamos un ejemplo de una imagen de 3x3 pixeles de distintos colores y pongamosle un numero distinto para cada color:

<div style="text-align:center">
<img src="img/pixeles.png" width="500"/>
</div>


NOTA: Hay muchas formas de establecer que numero le corresponde a cada color pero eso no importa por el momento.

Sabiendo esto podemos convertir una imagen a un vector (fila de numeros ordenados) de forma muy simple. Solamente ponemos uno despues de otro y esta imagen quedaria convertida en el siguiente vector:
(54,13,74,89,27,62,31,66,77)

Y nuestra tabla de perros y gatos quedaria de la siguiente forma:

<div style="text-align:center">
<img src="img/tabla_perros_gatos_vectores.png" width="700"/>
</div>

### Ejercicio 1:

Cargar una imagen de una banana madura y una normal en la notebook de jupyter usando la libreria PIL de python y convertir la imagen en:

1. Un vector de un solo elemento que es el promedio del color
2. Un vector de tres elementos con el promedio de R, G y B
3. Un vector aplanado de nxm elementos que son todos los pixeles de la imagen en una sola dimension

Entrenar un modelo de ML de la libreria sklearn con dichos vectores para poder clasificar las imagenes y ver los distintos resultados.

Utilizar las imagenes provistas en: 
- https://github.com/amiune/freecodingtour/tree/main/cursos/espanol/deeplearning/data/bananas

In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 1: Paso 1 {display-mode:"form"}

from PIL import Image
import numpy as np
import requests
from io import BytesIO

# Abre la imagen
response = requests.get("https://github.com/amiune/freecodingtour/raw/main/cursos/espanol/deeplearning/data/bananas/normal1.jpeg")
imagen = Image.open(BytesIO(response.content))
imagen

In [ ]:
#@title Solucion Ejercicio 1: Paso 2 {display-mode:"form"}

# Convierte la imagen en un arreglo NumPy
imagen_array = np.array(imagen)
imagen_array

In [ ]:
#@title Solucion Ejercicio 1: Paso 3 {display-mode:"form"}

# Calcula el promedio de todos los píxeles
promedio_pixeles = np.mean(imagen_array)
promedio_pixeles

In [ ]:
#@title Solucion Ejercicio 1: Paso 4 {display-mode:"form"}

# Calcula el promedio de cada canal de color (R, G y B)
promedio_canal_R = np.mean(imagen_array[:, :, 0])
promedio_canal_G = np.mean(imagen_array[:, :, 1])
promedio_canal_B = np.mean(imagen_array[:, :, 2])
promedio_canal_R,promedio_canal_G,promedio_canal_B

In [ ]:
#@title Solucion Ejercicio 1: Paso 5 {display-mode:"form"}

# Aplana el arreglo en un vector
imagen_vector_plano = imagen_array.flatten()
imagen_vector_plano

In [ ]:
#@title Solucion Ejercicio 1: Paso 6 {display-mode:"form"}

# Ahora repetiremos todos estos pasos para cada una de las imagenes
# y guardaremos los vectores resultantes en un diccionario

# Url donde se encuentran las imagenes
PATH = 'https://github.com/amiune/freecodingtour/raw/main/cursos/espanol/deeplearning/data/bananas/'
# Tamaño al que vamos a estandarizar a todas las imagenes
IMAGE_SIZE = (256, 256)

# Diccionario para guardar los resultados
vectores = {}

# Lista de nombres de las imagenes
nombres_imagenes = [f'normal{i}.jpeg' for i in range(1,6)]
nombres_imagenes.extend([f'madura{i}.jpeg' for i in range(1,6)])

for nombre_imagen in nombres_imagenes:
    # Cargar la imagen
    response = requests.get(PATH + nombre_imagen)
    
    with Image.open(BytesIO(response.content)) as img:

        # Hacer las imagenes del mismo tamaño
        img = img.resize(IMAGE_SIZE, Image.Resampling.LANCZOS)

        # Convertir a numpy array
        img_array = np.array(img)

        # Identificar los pixeles que no sean blancos
        non_blank_mask = (img_array[:,:,0] != 255) & (img_array[:,:,1] != 255) & (img_array[:,:,2] != 255)
        
        # Calcular el color promedio
        avg_color = np.mean(img_array[non_blank_mask][:,:])

        # Calcular el color promedio por canal
        avg_rgb = [np.mean(img_array[non_blank_mask][:,i]) for i in range(3)]

        # Aplanar la matriz
        flattened = img_array[:,:,:].flatten()

        # Guardar los resultados
        vectores[nombre_imagen] = {
            'avg_color': [avg_color],
            'avg_rgb': avg_rgb,
            'flattened': flattened,
            'clase': nombre_imagen[:6]
        }

vectores  # Mostrar los resultados

In [ ]:
#@title Solucion Ejercicio 1: Paso 7 {display-mode:"form"}

X = np.array([value['avg_rgb'] for key, value in vectores.items()])
y = np.array([value['clase'] for key, value in vectores.items()])
X, y

In [ ]:
#@title Solucion Ejercicio 1: Paso 7 [opcional] {display-mode:"form"}

# Transformar los datos a un dataframe de pandas
import pandas as pd
df = pd.DataFrame(data=X,columns=["R","G","B"])
df["clase"] = y
df

In [ ]:
#@title Solucion Ejercicio 1: Paso 7 [opcional] {display-mode:"form"}

# Mostrar los vectores con  un scatter plot
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# axes instance
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig)
fig.add_axes(ax)

# plot
colores = ['red' if x == 'madura' else 'blue' for x in df.clase]
sc = ax.scatter(df.R, df.G, df.B, c=colores, marker='o')
ax.set_xlabel('R Label')
ax.set_ylabel('G Label')
ax.set_zlabel('B Label');

# save
#plt.savefig("scatter_hue", bbox_inches='tight')

In [ ]:
#@title Solucion Ejercicio 1: Paso 8 {display-mode:"form"}

from sklearn.model_selection import train_test_split

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test

In [ ]:
#@title Solucion Ejercicio 1: Paso 9 {display-mode:"form"}

from sklearn.metrics import accuracy_score

# Predecir
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Real: ", y_test)
print("Prediccion: ", y_pred)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Si bien esta tecnica puede funcionar para casos muy simples como detectar el color de una banana, lamentablemente esta forma de convertir las imagenes a vectores no funciona muy bien en imagenes mas complejas para entrenar a nuestras cajas negras porque los pixeles se relacionan entre si para crear bordes, ojos, patas, etc y al ponerlos por separado las cajas negras no logran captar estas relaciones pero los cientificos empezaron a probar distintas formas de convertir las imagenes en vectores a partir de los cuales las cajas negras puedan aprender.

Un ejemplo de estas pruebas fue la aplicacion de filtros a las imagenes. Un filtro en particular es el de deteccion de bordes que convierte una imagen de la siguiente manera:


<div style="text-align:center">
<img src="img/deteccion_bordes.png" width="500" alt="Imagen de wikipedia"/>
</div>


Estos filtros se llaman de convolucion y se consiguen multiplicando los pixeles por matrices. Este proceso es el que les da el nombre a las famosas redes convolucionales.

<div style="text-align:center">
<img src="img/2D_Convolution_Animation.gif" width="500" alt="Imagen de wikipedia"/>
</div>

Luego se creo otra arquitectura, Vision Transformers, que tambien permite transformar imagenes a vectores de manera muy efectiva. Esta arquitectura se basa en lo descubierto al intentar transformar texto a vectores.

Por el momento existen estas dos arquitecturas que funcionan bien para convertir imagenes en vectores:
- Redes Convolucionales
- Redes Tranformers

Y en cada una de estas arquitecturas existen muchas variaciones.

Ambas arquitecturas realizan un proceso similar aunque de distinta forma y es el de encontrar formas y propiedades particulares en las imagenes para generar vectores que se podria decir que tienen una forma similar a la siguiente:

<div style="text-align:center">
<img src="img/tabla_perros_gatos_vectores2.png" width="700"/>
</div>

Afortunadamente todo este proceso que llevo muchos años de prueba y error fue introducido adentro de las cajas negras y estas cajas negras mas complejas que aprenden solas a como convertir las imagenes en vectores para poder aprender se llaman cajas negras de deep learning.

Probablemente en el futuro se sigan creando nuevas arquitecturas para convertir imagenes en vectores que sean aun mejores que las mencionadas.

Si quieres profundizar mas en como esta compuesta la arquitectura interna de dichas cajas negras hay mas informacion en las unidades avanzadas de este curso pero no es necesario conocer en profundidad como funcionan para poder utilizarlas.

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)

Dos imagenes de esta unidad fueron obtenidas de wikipedia:

- https://en.wikipedia.org/wiki/Edge_detection
- https://en.wikipedia.org/wiki/Kernel_(image_processing)